AUTHORS: Max Vincent and Noah Cabral

# Disclaimer

This Jupyter notebook is not meant to be run on Google Colab.
It will only run on a local machine.

In [7]:
# import things.
import os
import pickle
import imageio
import numpy as np
import time

In [8]:
# This function assumes that out_dir exists with subfolders 'evaluation' and 'training', each of
# which has subfolder 'color'.
# NOTE(Noah): Set is either "evaluation" or "training"
def parse_dataset(set, total_training_examples, dataset_dir, out_dir):
  path = os.path.join(dataset_dir, set)
  with open(os.path.join(path, 'anno_%s.pickle' % set), 'rb') as fi:
      anno_all = pickle.load(fi)
  valid_training_examples = 0
  start_time = time.time()
  path2 = out_dir
  print("Begin single hand parse")
  for sample_id,anno in anno_all.items():
    if sample_id > total_training_examples:
      break
    # format of the kp_visible array
    '''
    # 0: left wrist, 1-4: left thumb [tip to palm], 5-8: left index, ..., 17-20: left pinky,
    # 21: right wrist, 22-25: right thumb, ..., 38-41: right pinky
    '''
    kp_visible = (anno['uv_vis'][:, 2] == 1)
    case1 = np.sum(kp_visible[0:21])
    case2 = np.sum(kp_visible[21:])

    # NOTE(Noah): What cases are we invalidating?
    #   cases where none of the hands are seen.
    #   cases where two hands can be seen.
    #   images that contain left hands.
    one_hand = (case1 > 0 and case2 == 0) or (case1 == 0 and case2 > 0)
    rightHand = case2 > 0
    valid_case = one_hand and rightHand
    if (valid_case):
      valid_training_examples += 1
      image = imageio.imread(os.path.join(path, 'color', '%.5d.png' % sample_id))        
      filename = "%.5d.png" % sample_id
      print(filename)
      imageio.imwrite(os.path.join(path2,set,'color',filename),image)
            
  end_time = time.time()
  print("Total elapsed time for parse =", round(end_time - start_time, 2), "s")
  print("valid %s count:" % set, valid_training_examples)
  # print("Amount of %s training examples = " % set, round(valid_training_examples / total_training_examples * 100, 2), "%")

In [ ]:
# TODO(Noah): These constants here are duplicates from src/HandTracking.ipynb.
# we will want to modularize this code. 
TRAIN_TOTAL_COUNT = 41258
EVALUATION_TOTAL_COUNT = 2728

# NOTE: These are template lines. When doing actual dataset parsing, 
# we will want to change this.
dataset_dir = os.path.join("..", "RHD_small")
TRAIN_TOTAL_COUNT = 203
EVALUATION_TOTAL_COUNT = 203

# NOTE: This is also a template line. When doing actual dataset parsing, 
# we will want to change this.
out_dir = os.path.join("SHR_RHD")

os.mkdir(out_dir)
os.mkdir(os.path.join(out_dir, "evaluation"))
os.mkdir(os.path.join(out_dir, "training"))
os.mkdir(os.path.join(out_dir, "evaluation", "color"))
os.mkdir(os.path.join(out_dir, "training", "color"))

set = "evaluation"
parse_dataset(set, EVALUATION_TOTAL_COUNT, dataset_dir, out_dir)

set = "training"
parse_dataset(set, TRAIN_TOTAL_COUNT, dataset_dir, out_dir)